In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import  LinearSVR
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score

In [ ]:
data = pd.read_csv('./train_V2.csv')
data.head()

In [ ]:
def prepare(X):
    dct = {}
    for a, b, c in zip(X.matchId, X.damageDealt, range(X.shape[0])):
        if a not in dct:
            dct[a] = []
        dct[a].append((b, c))
    X['dd_place'] = np.zeros(X.shape[0])
    for x in dct.values():
        x.sort()
        for i in range(len(x)):
            X['dd_place'].at[x[i][1]] = i
    X['boosts+heals'] = X['boosts'] + X['heals']
    X['headshot_rate'] = X['headshotKills'] / X['kills']
    X['headshot_rate'] = X['headshot_rate'].fillna(0)
    X['groupId'] = X['groupId'].astype('category')
    X['matchId'] = X['matchId'].astype('category')
    X['groupId_cat'] = X['groupId'].cat.codes
    X['matchId_cat'] = X['matchId'].cat.codes
    X.drop(columns=['groupId', 'matchId', 'Id'], inplace=True)
    X = pd.get_dummies(X, columns=['matchType'])
    X['rankPoints'] = X['rankPoints'].replace(-1, np.nan)
    X['winPoints'] = X['winPoints'].replace(0, np.nan)
    X.fillna(X.mean(), inplace=True)
#     X = StandardScaler().fit_transform(X)
    return X

In [ ]:
y_train = data.winPlacePerc
y_train.fillna(y_train.mean())
X_train = prepare(data.copy().drop(axis=1, columns=['winPlacePerc']))
y_train = y_train.fillna(y_train.mean())

In [ ]:
model = RandomForestRegressor(n_jobs=-1)

In [ ]:
# -cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error').mean()

In [ ]:
data_test = pd.read_csv('./test_V2.csv')
Id = list(data_test['Id'])
X_test = prepare(data_test.copy())
y_pred = model.fit(X_train, y_train).predict(X_test)
pd.DataFrame({'Id' : Id, 'winPlacePerc' : y_pred}).to_csv('sample_submission.csv', index=False)

# Что вообще тут происходит

1. Для каждого матча сортируем игроков по урону, который он нанес(гипотеза - если чел нанес ооочень много урона про другим, то он, наверное, крут). А если почти не нанес, то скорее всего он умер в самом начале.
2. Считаем "меткость стрелка" - частота хедшотов.
3. boosts ~= heals, поэтому го считать и отдельно, и вместе.
4. Переводим Id в категории - говорят, что это может быть полезно
5. Смотрим на тип матча, делаем OH
6. Заменяем все плохие значения на среднее
7. Запускаем на этом случайный лес

Скор - 0.0604